# Clustering

See:
* [Alexie's 214 notes](https://github.com/alexieleauthaud/ASTR214_2017/blob/master/class9and10_clustering.ipynb)
* [Corrfunc, a fast implementation of these algos](https://github.com/manodeep/Corrfunc)
* [Appendix A in Zehavi+ 2011](https://iopscience.iop.org/article/10.1088/0004-637X/736/1/59/meta#apj391584app1) talks about the information content of auto vs cross correlation.

Most broadly, what is the probability of finding an object in a volume $dV$ some distance $r$ away from another object? 

# Two Point Correlation Function (2pcf)

What is the excess probability (compared to a random distribution) of finding two galaxies separated by a distance $r$. **Excess** is important, else you are also measuring number density $n$. Mathematically,

$$
dP(r) = n_1 n_2(1 + \xi(r))dV_1 dV_2
$$

where $\xi(r)$ is the correlation function which is unitless and must be $> -1$ (else probability goes negative and we can't have that!). $r$ is the vector pointing from $V_1$ to $V_2$. However, as we assume things are isotropic, only the length of that vector matters.

We know that on average the probability of finding a galaxiy in a volumne element is $dP = n dV$ and so $\xi(r) > 0$ implies clustering - galaxies are more likely to be separated by that distance than random. $\xi(r) < 0$ implies anti-clustering.

To compute this in practice we setup bins, e.g. $10 < r < 11$Mpc. We then count the number of pairs that are separated by a length that fits in that bin. If we are working in a simulation we can then use the known $n$ and volume to compare this count to the expected one. However, in observations with complex selection effects (redshift dependences, masks, collisions, etc) a carefully constructed random catalog should be used to find the expected number of pairs with this separation.

## Auto vs Cross correlation

Which objects are we measuring the clustering of? There are two options:
* Auto correlation: Select a sample of objects (e.g. galaxies with stellar mass between $10^{11}$ and $10^{11.1}$) and find the excess probability of finding members of this sample at a distance $r$ away from other members it.
* Cross correlation: Select two samples of objects (e.g. galaxies with stellar mass between $10^{11}$ and $10^{11.1}$ (sample 1) and ones between $10^{11.1}$ and $10^{11.2}$ (sample 2)) and find the excess probability of finding members of sample 2 at a distance $r$ from sample 1. Note that the order of these doesn't matter.

## 2d vs 3d correlation

Imagine you don't have redshifts, you only have angular (RA, Dec) positions on the sky. You can still define a 2d angular correlation function,

$$
dP = n_1 n_2 (1 + \omega(\theta))dA_1 dA_2
$$

where $n$ is now the projected mean number density and $\omega(\theta)$ is the angular correlation function.

There are many other correlation functions we can build. If we define:
* $r$: Separation in real space
* $s$: Separation in redshift space
* $r_p$: Redshift space separation perpendicular to the LOS 
* $r_{\pi}$: Redshift space separation along the LOS 
* $\theta$: Angular separation

We can have:
* $\xi(r)$: Real space 3d correlation function
* $\xi(s)$: Redshift space 3d correlation function. Sometimes called $\xi(r_p, r_{\pi})$
* $\omega_p(r_p)$: Correlation as a function of perpendicular separation (projected)
* $\omega(\theta)$: The angular version of $\omega_p(r_p)$, not using redshift to get distances

# Total information content 

Let's say we have a population of galaxies A(ll) that we divide into two populations, B(lue) and R(ed). We can compute 5 correlation functions from these populations: AA, BB, RR, BR, RB. Do all of these contain independent information?

What is the probability that we find *any* pair at a given separation? Somewhat trivially, it must just be the sum of the probabilities of finding each of the 3 types of pairs that it could be.

$$
dP_A = dP_B + dP_R + dP_{BR} + dP_{RB}
$$

Of course, $dP_{BR} = dP_{RB}$ and so,

$$
dP_A = dP_B + dP_R + 2 dP_{BR}
$$

Therefore in terms of $\xi$,

$$
n_A^2(1 + \xi_A) = n_B^2 (1 + \xi_B) + n_R^2 (1 + \xi_R) + 2 n_B n_R (1 + \xi_{BR})
$$

but, $n_A^2 = (n_B + n_R)^2 = n_B^2 + n_R^2 + 2 n_B n_R$ and so,

$$
n_A^2 \xi_A = n_B^2 \xi_B + n_R^2 \xi_R + 2 n_B n_R \xi_{BR}
$$

And therefore given just 3 of these we can determine the fourth.
